In [1]:
pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install flask

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
cd /content/drive/MyDrive/hackfest_intent

/content/drive/MyDrive/hackfest_intent


In [5]:
import numpy as np
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer
from flask import Flask, request, render_template
from flask_ngrok import run_with_ngrok
import pickle

from torch import cuda

In [6]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!ngrok authtoken 2OA8UNAuBk4mn4gwyWtp6kRdNAO_6ceuyQ6TwNDEnNrERQXFP

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [8]:
class DistillBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 38)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [9]:
device = 'cuda' if cuda.is_available() else 'cpu'

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')

model = pickle.load(open('model.pkl', 'rb'))

classes =['transactions', 'thank_you', 'reminder_update', 'meeting_schedule', 'to-do list', 'order', 'make_call', 
          'credit_limit', 'exchange_rate', 'insurance', 'international_fees', 'last_maintenance', 
          'insurance_change', 'application_status', 'date', 'schedule_meeting', 'pin_change', 'expiration_date', 
          'are_you_a_bot', 'card_declined', 'credit_score', 'new_card', 'income', 'calendar', 'next_holiday', 'user name',
         'pay_bill','freeze account', 'balance', 'replacement_card_duration', 'schedule_maintenance', 'improve_credit_score',
           'taxes', 'bill_balance', 'credit_limit_change', 'account_blocked', 'transfer', 'order_checks', 'interest_rate', 
           'bill_due', 'rewards_balance', 'direct_deposit','change_language', 'change_user_name', 'spending_history' ]


def get_intent(event, max_length=512):
  input_text = "%s </s>" % event
  text = input_text
  text = " ".join(text.split())
  inputs = tokenizer.encode_plus(
      text,
      None,
      add_special_tokens=True,
      max_length=max_length,
      pad_to_max_length=True,
      return_token_type_ids=True,
      truncation=True
  )

  ids= torch.tensor(inputs['input_ids'], dtype=torch.long)
  mask = torch.tensor(inputs['attention_mask'], dtype=torch.long)

  ids = ids.to(device, dtype = torch.long)
  mask = mask.to(device, dtype = torch.long)

  outputs = model(ids, mask).squeeze()
  return classes[torch.argmax(outputs).item()]

app = Flask(__name__)

run_with_ngrok(app)

@app.route("/")
def home():
    return render_template("intent.html")

@app.route('/intent', methods=['POST'])
def res():
  text = request.form["string-input"]
  prediction = get_intent(text)

  return render_template('IntentResult.html', event = text, result = prediction)

In [10]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://f0e6-35-225-33-217.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [08/Apr/2023 22:27:25] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Apr/2023 22:27:27] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [08/Apr/2023 22:28:06] "GET / HTTP/1.1" 200 -
